In [104]:
import pandas as pd
colspecs = [(0, 2), (20, 30), (31, 59), (60, 160),(161,164),(165,194),(195,197),(198,205),(206,235),(236,288),(288,290),(290,292),(361,364),(364,377),(394,460)]
update=pd.read_fwf('fwso/sample txtfile.txt',
                   header=None, 
                   colspecs=colspecs,
                   names=['dir_code','mem_wstd','first_name','last_name','sam_city','sam_stname','sam_stsubt','sam_stnmfr','bla','ble','listcode','acc_type','regulatory_id','acct_no','dist_code']
                  ,)
#update.()
update.sort_values(by='last_name',inplace=True)
print update

  dir_code  mem_wstd         first_name  \
1       BL   2236040     ARLEEN  GARCIA   
4       BL   2236059                NaN   
6       BL   2291961                NaN   
2       BL   2236049  ARRIANE  ARGAMOSA   
3       BL   2236054       FE  FENANDEZ   
0       BL   2236022   ISAGANI  DE LEON   
8       BL   2292204                NaN   
9       BL   2292205                NaN   
5       BL   2282409                NaN   
7       BL   2292203                NaN   

                                last_name sam_city        sam_stname  \
1                               CRISTOBAL      OBA           TAWIRAN   
4                       JOCELYN FORGE INC      MEY         BAGBAGUIN   
6    L.G RAMOS TRADING AND MARINESERVICES      OBA       F NAVARETTE   
2                                 LAUREAN      MEY             TOPAZ   
3                                 LORENZO      OBA  PRES S OSMENA SR   
0                                  MARTIN      OBA          STO NINO   
8                OBAND

In [107]:
update.append(update).sort_values('last_name')

,dir_code,mem_wstd,first_name,last_name,sam_city,sam_stname,sam_stsubt,sam_stnmfr,bla,ble,listcode,acc_type,regulatory_id,acct_no,dist_code
1,BL,2236040,ARLEEN GARCIA,CRISTOBAL,OBA,TAWIRAN,RD,45.0,NaN,NaN,PB,RR,PLD,241336611,BULACAN OBANDO
1,BL,2236040,ARLEEN GARCIA,CRISTOBAL,OBA,TAWIRAN,RD,45.0,NaN,NaN,PB,RR,PLD,241336611,BULACAN OBANDO
4,BL,2236059,NaN,JOCELYN FORGE INC,MEY,BAGBAGUIN,RD,56.0,NaN,NaN,PB,BR,PLD,161375209,BULACAN MEYCAUAYAN CITY
4,BL,2236059,NaN,JOCELYN FORGE INC,MEY,BAGBAGUIN,RD,56.0,NaN,NaN,PB,BR,PLD,161375209,BULACAN MEYCAUAYAN CITY
6,BL,2291961,NaN,L.G RAMOS TRADING AND MARINESERVICES,OBA,F NAVARETTE,ST,25.0,NaN,NaN,PB,BR,PLD,212837784,BULACAN OBANDO
6,BL,2291961,NaN,L.G RAMOS TRADING AND MARINESERVICES,OBA,F NAVARETTE,ST,25.0,NaN,NaN,PB,BR,PLD,212837784,BULACAN OBANDO
2,BL,2236049,ARRIANE ARGAMOSA,LAUREAN,MEY,TOPAZ,ST,NaN,CITATION HOMES,NaN,PB,RR,PLD,217398665,BULACAN MEYCAUAYAN CITY
2,BL,2236049,ARRIANE ARGAMOSA,LAUREAN,MEY,TOPAZ,ST,NaN,CITATION HOMES,NaN,PB,RR,PLD,217398665,BULACAN MEYCAUAYAN CITY
3,BL,2236054,FE FENANDEZ,LORENZO,OBA,PRES S OSMENA SR,ST,523.0,NaN,NaN,PB,RR,PLD,244072801,BULACAN OBANDO
3,BL,2236054,FE FENANDEZ,LORENZO,OBA,PRES S OSMENA SR,ST,523.0,NaN,NaN,PB,RR,PLD,244072801,BULACAN OBANDO
